In [1]:
from fastai.tabular.all import *

In [2]:
train=pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
test=pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")

submission=pd.read_csv("/kaggle/input/playground-series-s4e5/sample_submission.csv")

original=pd.read_csv("/kaggle/input/flood-prediction-factors/flood.csv")

train.drop(columns=["id"],inplace=True)
test.drop(columns=["id"],inplace=True)

train=pd.concat([train,original],axis='rows')
train.reset_index(inplace=True,drop=True)

target='FloodProbability'

In [3]:
train.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,5,8,5,8,6,4,4,3,3,4,...,5,3,3,5,4,7,5,7,3,0.445
1,6,7,4,4,8,8,3,5,4,6,...,7,2,0,3,5,3,3,4,3,0.450
2,6,5,6,7,3,7,1,5,4,5,...,7,3,7,5,6,8,2,3,3,0.530
3,3,4,6,5,4,8,4,7,6,8,...,2,4,7,4,4,6,5,7,5,0.535
4,5,3,2,6,4,4,3,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415


In [4]:
def meta_features(df, initial_features ):
    
    df[original.drop(columns=[target]).columns]=df[original.drop(columns=[target]).columns].astype(np.uint8)
    df['fsum'] = df[initial_features].sum(axis=1) # for tree models
    print('comnputing f_std')
    df['f_std']  = df[initial_features].std(axis=1)
    print('comnputing f_mean')
    df['f_mean'] = df[initial_features].mean(axis=1)
    print('comnputing f_max')
    df['f_max']  = df[initial_features].max(axis=1)
    print('comnputing f_min')
    df['f_min']  = df[initial_features].min(axis=1)
#     print('comnputing f_mode')
#     df['f_mode'] = df[initial_features].mode(axis=1)[0]
    print('comnputing f_median')
    df['f_median'] = df[initial_features].median(axis=1)
    print('comnputing f_25th')
    df['f_25th'] = df[initial_features].quantile(0.25, axis=1)
    print('comnputing f_75th')
    df['f_75th'] = df[initial_features].quantile(0.75, axis=1)
    print('comnputing f_skew')
    df['f_skew'] = df[initial_features].skew(axis=1)
    print('comnputing f_kurt')
    df['f_kurt'] = df[initial_features].kurt(axis=1)
    df['special1'] = df['fsum'].isin(np.arange(72, 76)) # for linear models
    for i in range(10,100,10):
        print(f'comnputing f_{i}th')
        df[f'f_{i}th'] = df[initial_features].quantile(i/100, axis=1)
    print('comnputing f_harmonic')
    df['f_harmonic'] = len(initial_features) / df[initial_features].apply(lambda x: (1/x).mean(), axis=1)
    print('comnputing f_geometric')
    df['f_geometric'] = df[initial_features].apply(lambda x: x.prod()**(1/len(x)), axis=1)
    print('comnputing f_zscore')
    df['f_zscore'] = df[initial_features].apply(lambda x: (x - x.mean()) / x.std(), axis=1).mean(axis=1)
    print('computing Coefficient of Variation ')
    df['f_cv'] = df[initial_features].std(axis=1) / df[initial_features].mean(axis=1)
    print('computing f_Quantile Coefficients of Skewness_75')
    df['f_Quantile Coefficients of Skewness_75'] = (df[initial_features].quantile(0.75, axis=1) - df[initial_features].mean(axis=1)) / df[initial_features].std(axis=1)
    print('computing f_Quantile Coefficients of Skewness_25')
    df['f_Quantile Coefficients of Skewness_25'] = (df[initial_features].quantile(0.25, axis=1) - df[initial_features].mean(axis=1)) / df[initial_features].std(axis=1)
    print('computing f_2ndMoment')
    df['f_2ndMoment'] = df[initial_features].apply(lambda x: (x**2).mean(), axis=1)
    print('computing f_3rdMoment')
    df['f_3rdMoment'] = df[initial_features].apply(lambda x: (x**3).mean(), axis=1)
    print('computing f_entropy')
    df['f_entropy'] = df[initial_features].apply(lambda x: -1*(x*np.log(x)).sum(), axis=1)
    
    return df

initial_features=test.columns
train=meta_features(train,initial_features )
test =meta_features(test, initial_features)

comnputing f_std
comnputing f_mean
comnputing f_max
comnputing f_min
comnputing f_median
comnputing f_25th
comnputing f_75th
comnputing f_skew
comnputing f_kurt
comnputing f_10th
comnputing f_20th
comnputing f_30th
comnputing f_40th
comnputing f_50th
comnputing f_60th
comnputing f_70th
comnputing f_80th
comnputing f_90th
comnputing f_harmonic
comnputing f_geometric
comnputing f_zscore
computing Coefficient of Variation 
computing f_Quantile Coefficients of Skewness_75
computing f_Quantile Coefficients of Skewness_25
computing f_2ndMoment
computing f_3rdMoment
computing f_entropy


/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


comnputing f_std
comnputing f_mean
comnputing f_max
comnputing f_min
comnputing f_median
comnputing f_25th
comnputing f_75th
comnputing f_skew
comnputing f_kurt
comnputing f_10th
comnputing f_20th
comnputing f_30th
comnputing f_40th
comnputing f_50th
comnputing f_60th
comnputing f_70th
comnputing f_80th
comnputing f_90th
comnputing f_harmonic
comnputing f_geometric
comnputing f_zscore
computing Coefficient of Variation 
computing f_Quantile Coefficients of Skewness_75
computing f_Quantile Coefficients of Skewness_25
computing f_2ndMoment
computing f_3rdMoment
computing f_entropy


/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
train.to_csv("/kaggle/working/train_meta_f.csv")
test.to_csv("/kaggle/working/test_meta_f.csv")

In [6]:
train.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,f_90th,f_harmonic,f_geometric,f_zscore,f_cv,f_Quantile Coefficients of Skewness_75,f_Quantile Coefficients of Skewness_25,f_2ndMoment,f_3rdMoment,f_entropy
0,5,8,5,8,6,4,4,3,3,4,...,7.1,82.151589,4.397569,-7.771561e-17,0.372380,0.314252,-0.971324,25.00,70.70,-151.625
1,6,7,4,4,8,8,3,5,4,6,...,8.0,0.000000,0.000000,-6.661338e-17,0.488606,0.674955,-0.740273,27.10,73.60,-158.000
2,6,5,6,7,3,7,1,5,4,5,...,7.0,76.242342,4.468658,-8.881784e-17,0.390386,0.672735,-1.009102,28.05,94.65,-166.500
3,3,4,6,5,4,8,4,7,6,8,...,7.1,92.766427,4.933055,-1.089406e-16,0.315686,0.639633,-0.731010,29.60,91.40,-176.500
4,5,3,2,6,4,4,3,3,3,3,...,6.0,58.394161,3.275062,-9.992007e-17,0.416910,0.932788,-0.566336,15.10,70.50,-98.375


In [7]:
cont_names,cat_names = cont_cat_split(train, dep_var='FloodProbability')
splits = RandomSplitter(valid_pct=0.2)(range_of(train))


In [8]:
to = TabularPandas(train, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='FloodProbability',
                   y_block=RegressionBlock(),
                   splits=splits)

In [9]:
dls = to.dataloaders(bs=1024)
test_dl = dls.test_dl(test)

In [10]:
# dls = TabularDataLoaders.from_df(df, y_names="FloodProbability",bs=1024)

In [11]:
# dls.show_batch()

In [12]:
learn = tabular_learner(dls, metrics=[accuracy,R2Score()],lr=0.01)

In [13]:
# suggest_funcs = (minimum, steep, valley, slide)
# learn.lr_find(suggest_funcs=suggest_funcs)

In [14]:
learn.fine_tune(12)

epoch,train_loss,valid_loss,accuracy,r2_score,time
0,0.000779,0.000519,0.000000,0.801736,00:17


epoch,train_loss,valid_loss,accuracy,r2_score,time
0,0.000432,0.000432,0.000000,0.834779,00:17
1,0.000430,0.000393,0.000000,0.849801,00:17
2,0.000426,0.000434,0.000000,0.834185,00:17
3,0.000440,0.000380,0.000000,0.854684,00:17
4,0.000413,0.000402,0.000000,0.846570,00:17
5,0.000388,0.000424,0.000000,0.837862,00:17
6,0.000383,0.000388,0.000000,0.851887,00:17
7,0.000369,0.000366,0.000000,0.860076,00:17
8,0.000357,0.000353,0.000000,0.865233,00:17
9,0.000355,0.000359,0.000000,0.862918,00:17


In [15]:
# test_df = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv',index_col='id')
# dl = learn.dls.test_dl(test_df)

In [16]:
preds = learn.get_preds(dl=test_dl)
preds

(tensor([[0.5762],
         [0.4556],
         [0.4529],
         ...,
         [0.6201],
         [0.5538],
         [0.5281]]),
 None)

In [17]:
preds = preds[0].cpu().numpy()

In [18]:
sub = pd.read_csv("/kaggle/input/playground-series-s4e5/sample_submission.csv")
sub['FloodProbability'] = preds
sub.head()

,id,FloodProbability
0,1117957,0.576213
1,1117958,0.455625
2,1117959,0.452855
3,1117960,0.467433
4,1117961,0.469709


In [19]:
sub.to_csv("/kaggle/working/submission.csv",index=False)